# Slight Detour: What is Semantic Similarity?

Semantic similarity is a concept in natural language processing and computational linguistics that measures how similar two pieces of text are in meaning, rather than just looking at lexical (word-level) matches.

At its core, semantic similarity aims to capture when two different texts express similar ideas, concepts, or information, even if they use completely different words. For example, "The automobile won't start" and "My car isn't working" have high semantic similarity despite using different vocabulary.

![Semantic SImilarity](./images/semantic_similarity.png "Semantic Similarity")

In [ ]:
! pip install faiss-cpu

In [1]:
import requests
import numpy as np
import faiss

In [2]:
texts = [
    "The cat sat on the mat",
    "A feline was resting on a rug",
    "Dogs are great pets",
    "I love having a canine companion",
    "Paris is the capital of France",
    "The Eiffel Tower is in Paris"
]

In [4]:
# Using Nomic model served locally via Ollama for embedding
# Ollama is a friend --> https://ollama.com/
def get_embeddings_from_ollama(text, model="nomic-embed-text"):
    url = "http://localhost:11434/api/embeddings"
    
    payload = {
        "model": model,
        "prompt": text
    }
    
    response = requests.post(url, json=payload)
    return np.array(response.json()["embedding"], dtype=np.float32)


In [5]:
# Generate embeddings for all texts
embeddings = []
for text in texts:
    embedding = get_embeddings_from_ollama(text)
    embeddings.append(embedding)

In [6]:
# Convert list to numpy array and create FAISS index
embeddings = np.array(embeddings)

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

In [17]:
# Create an embedding for input 
# We will compare this sentence to the ones above
query_text = "A dog sat on a mat"
query_vector = get_embeddings_from_ollama(query_text)
query_vector = query_vector.reshape(1, -1)  # Reshape for FAISS

In [18]:
# Print out for fun
print(f"Vector shape: {query_vector.shape}")
print(f"Vector type: {query_vector.dtype}")
print(query_vector)

Vector shape: (1, 768)
Vector type: float32
[[ 4.14752126e-01  5.60575545e-01 -2.99866676e+00 -9.46711540e-01
   6.69734955e-01  1.23252892e+00 -4.54351246e-01  1.25170910e+00
  -1.16572595e+00 -1.81598485e+00  2.25684404e-01  1.06520557e+00
   8.25459838e-01 -4.00354862e-01 -3.10114682e-01 -1.24684858e+00
  -2.66173899e-01 -1.30672324e+00  1.18963814e+00 -7.31207073e-01
   3.21554691e-01  7.20677137e-01  1.22334741e-01  7.64824510e-01
   1.73462784e+00  2.17485279e-01  7.35670328e-01  1.75638247e+00
   1.98791170e+00  7.01488554e-01  3.11943501e-01 -1.70698568e-01
  -6.85368121e-01 -1.04187381e+00  3.30846518e-01  1.46218166e-01
   1.02840400e+00  2.22483969e+00  3.18641782e-01  1.16057180e-01
  -3.82416070e-01 -8.95320237e-01  6.44932315e-02  4.82710987e-01
   7.64767289e-01 -1.60909986e+00  1.07649148e-01  3.46771628e-01
  -6.80581927e-01 -3.75585884e-01  4.67031747e-01  2.86214471e-01
  -1.05138347e-02 -1.07734418e+00  1.41030455e+00  1.06990361e+00
   1.62347746e+00  1.47903532e-0

In [19]:
# Search for similarity with ALL texts in the index
k = len(texts)  # Return all results
distances, indices = index.search(query_vector, k)

# Display results
print(f"Query: {query_text}\n")
print("All texts ranked by similarity:")
for i, (dist, idx) in enumerate(zip(distances[0], indices[0])):
    print(f"{i+1}. \"{texts[idx]}\" (Distance: {dist:.4f})")

Query: A dog sat on a mat

All texts ranked by similarity:
1. "The cat sat on the mat" (Distance: 143.3939)
2. "A feline was resting on a rug" (Distance: 268.1303)
3. "Dogs are great pets" (Distance: 411.7034)
4. "I love having a canine companion" (Distance: 538.4823)
5. "The Eiffel Tower is in Paris" (Distance: 607.5668)
6. "Paris is the capital of France" (Distance: 648.7864)
